In [1]:
import numpy as np
import pandas as pd
import string
import math
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 400)

In [2]:
file_name = "Tab.delimited.Cleaned.dataset.WITH.variable.labels"
df = pd.read_csv(r"ML1/" + file_name + ".preprocessed1" + ".csv", sep=',', encoding='utf-8', low_memory=False)

In [3]:
import math
def mutual_info(df):
  mutual_info_matrix = np.zeros(shape = (df.shape[1], df.shape[1]))
  for i, col1 in enumerate(list(df)):
    for j, col2 in enumerate(list(df)):
      if j < i:
        continue
#       print("cols ", col1, " ", col2)
      prob1 = {}
      prob2 = {}
      prob12 = {}
      for val1 in df[col1].unique():
        if pd.isnull(val1):
          prob1["nan"] = (df[col1].isnull().value_counts()[True]/df.shape[0])
          continue
        prob1[val1] = (df[col1].value_counts()[val1]/df.shape[0])
#         print(val1, " ", prob1[val1])
#       print()
      for val2 in df[col2].unique():
        if pd.isnull(val2):
          prob2["nan"] = (df[col2].isnull().value_counts()[True]/df.shape[0])
          continue
        prob2[val2] = (df[col2].value_counts()[val2]/df.shape[0])
#         print(val2, " ", prob2[val2])
      for val1 in df[col1].unique():
#         if pd.isnull(val1):
#           continue
        for val2 in df[col2].unique():
          if pd.isnull(val1) and not pd.isnull(val2):
            prob12[("nan", val2)] = df[df[col1].isnull() & (df[col2] == val2)].shape[0]/df.shape[0]
          elif not pd.isnull(val1) and pd.isnull(val2):
            prob12[(val1, "nan")] = df[(df[col1] == val1) & df[col2].isnull()].shape[0]/df.shape[0]
          elif pd.isnull(val1) and pd.isnull(val2):
            prob12[("nan", "nan")] = df[df[col1].isnull() & df[col2].isnull()].shape[0]/df.shape[0]
          else:
            prob12[(val1, val2)] = df[(df[col1] == val1) & (df[col2] == val2)].shape[0]/df.shape[0]
#           print(val1,  " ", val2, " ", prob12[(val1, val2)])   
#       print()
      mutual_in = 0
      for k in range(df.shape[0]):
        if pd.isnull(df.iloc[k, i]) and not pd.isnull(df.iloc[k, j]):
          ratio = prob12[("nan", df.iloc[k, j])]/(prob1["nan"]*prob2[df.iloc[k, j]])
        elif not pd.isnull(df.iloc[k, i]) and pd.isnull(df.iloc[k, j]):
          ratio = prob12[(df.iloc[k, i], "nan")]/(prob1[df.iloc[k, i]]*prob2["nan"])
        elif pd.isnull(df.iloc[k, i]) and pd.isnull(df.iloc[k, j]):
          ratio = prob12[("nan", "nan")]/(prob1["nan"]*prob2["nan"])
        else:
          ratio = prob12[(df.iloc[k, i], df.iloc[k, j])]/(prob1[df.iloc[k, i]]*prob2[df.iloc[k, j]])
        if ratio == 0:
          continue
#         print("lol ", i, " ", j, " ", k)
#         print("p in both ", prob12[(df.iloc[k, i], df.iloc[k, j])], " p in i ", prob1[df.iloc[k, i]], " p in j ", prob2[df.iloc[k, j]])
        mutual_in = mutual_in + math.log(ratio)
      mutual_info_matrix[i][j] = mutual_in
      print("mutual info for ", i, " ", j, " ", mutual_in)
  return mutual_info_matrix

In [4]:
df_temp = pd.DataFrame(np.array([[0, 1, np.NaN], [0, np.NaN, 4], [1, 2, np.NaN], [0, 1, 6]]))

In [5]:
df_temp

,0,1,2
0,0.0,1.0,NaN
1,0.0,NaN,4.0
2,1.0,2.0,NaN
3,0.0,1.0,6.0


In [6]:
mutual_info(df_temp)

mutual info for  0   0   2.249340578475233
mutual info for  0   1   2.249340578475233
mutual info for  0   2   0.8630462173553426
mutual info for  1   1   4.1588830833596715
mutual info for  1   2   2.772588722239781
mutual info for  2   2   4.1588830833596715


array([[ 2.24934058,  2.24934058,  0.86304622],
       [ 0.        ,  4.15888308,  2.77258872],
       [ 0.        ,  0.        ,  4.15888308]])

In [4]:
mutual_info(df.iloc[:, 127:])

mutual info for  0   0   6311.218098033685
mutual info for  0   1   185.77867233701932
mutual info for  0   2   6.836053237244683
mutual info for  0   3   1.7229113303780472
mutual info for  0   4   1.2981340287652419
mutual info for  0   5   4.080571376693224
mutual info for  0   6   1.2278438591797785
mutual info for  0   7   5.6369446489210855
mutual info for  0   8   21.1893648663753
mutual info for  0   9   26.240233119902143
mutual info for  0   10   9.947703394668164
mutual info for  0   11   15.164416171436258
mutual info for  0   12   2.9425747456844307
mutual info for  0   13   20.611622059025105
mutual info for  0   14   8.883219217016329
mutual info for  0   15   6.023303007456846
mutual info for  0   16   8.787763786627696
mutual info for  0   17   1818.0068156813622
mutual info for  0   18   5.452445518422048
mutual info for  0   19   18.65188005063601
mutual info for  0   20   27.14177590380077
mutual info for  0   21   10.616562126566405
mutual info for  0   22   20.786

KeyboardInterrupt: 

In [7]:
df.shape

(6344, 205)

In [8]:
df.race.value_counts().sum()

6314

In [9]:
def calculate_info_gain_g(df, col, y, consider_nan = "True"):
  if consider_nan:
    y_value_counts = df.iloc[:, y].value_counts().to_dict()
    y_value_counts["nan"] = df.shape[0] - df.iloc[:, y].value_counts().sum()
    col_value_counts = df.iloc[:, col].value_counts().to_dict()
    col_value_counts["nan"] = df.shape[0] - df.iloc[:, col].value_counts().sum()
    h_y = 0
    h_x_y = 0
    count_dict = {}
    
    for val1 in df.iloc[:, col].unique():
      for val2 in df.iloc[:, y].unique():
        if pd.isnull(val1) and pd.isnull(val2):
          count_dict[("nan", "nan")] = np.count_nonzero((df.iloc[:, col].isnull()) & (df.iloc[:, y].isnull()))
        elif pd.isnull(val1) and not pd.isnull(val2):
          count_dict[("nan", val2)] = np.count_nonzero((df.iloc[:, col].isnull()) & (df.iloc[:, y] == val2))
        elif not pd.isnull(val1) and pd.isnull(val2):
          count_dict[(val1, "nan")] = np.count_nonzero((df.iloc[:, col] == val1) & (df.iloc[:, y].isnull()))
        elif not pd.isnull(val1) and not pd.isnull(val2):
          count_dict[(val1, val2)] = np.count_nonzero((df.iloc[:, col] == val1) & (df.iloc[:, y] == val2))
    for val in y_value_counts.keys():
      if y_value_counts[val] != 0:
        h_y = h_y - y_value_counts[val]/df.shape[0] * math.log(y_value_counts[val]/df.shape[0])
    
    for val1 in df.iloc[:, col].unique():
      for val2 in df.iloc[:, y].unique():
        if pd.isnull(val1) and pd.isnull(val2) and count_dict[("nan", "nan")] != 0:
          h_x_y = h_x_y - col_value_counts["nan"]/df.shape[0]*count_dict[("nan", "nan")]/col_value_counts["nan"]*\
          math.log(count_dict[("nan", "nan")]/col_value_counts["nan"])
        elif pd.isnull(val1) and not pd.isnull(val2) and count_dict[("nan", val2)] != 0:
          h_x_y = h_x_y - col_value_counts["nan"]/df.shape[0]*count_dict[("nan", val2)]/col_value_counts["nan"]*\
          math.log(count_dict[("nan", val2)]/col_value_counts["nan"])
        elif not pd.isnull(val1) and pd.isnull(val2) and count_dict[(val1, "nan")] != 0:
          h_x_y = h_x_y - col_value_counts[val1]/df.shape[0]*count_dict[(val1, "nan")]/col_value_counts[val1]*\
          math.log(count_dict[(val1, "nan")]/col_value_counts[val1])
        elif not pd.isnull(val1) and not pd.isnull(val2) and count_dict[(val1, val2)] != 0:
          h_x_y = h_x_y - col_value_counts[val1]/df.shape[0]*count_dict[(val1, val2)]/col_value_counts[val1]*\
          math.log(count_dict[(val1, val2)]/col_value_counts[val1])  
  else:
    y_value_counts = df.iloc[:, y].value_counts()
#     y_value_counts["nan"] = df.shape[0] - df.iloc[:, y].value_counts().sum()
    col_value_counts = df.iloc[:, col].value_counts()
#     col_value_counts["nan"] = df.shape[0] - df.iloc[:, col].value_counts().sum()
    h_y = 0
    h_x_y = 0
    count_dict = {}
    
    for val1 in df.iloc[:, col].unique():
      for val2 in df.iloc[:, y].unique():
        if pd.isnull(val1) and pd.isnull(val2):
#           count_dict[("nan", "nan")] = np.count_nonzero((df.iloc[:, col].isnull()) & (df.iloc[:, y].isnull()))
          pass
        elif pd.isnull(val1) and not pd.isnull(val2):
#           count_dict[("nan", val2)] = np.count_nonzero((df.iloc[:, col].isnull()) & (df.iloc[:, y] == val2))
          pass
        elif not pd.isnull(val1) and pd.isnull(val2):
#           count_dict[(val1, "nan")] = np.count_nonzero((df.iloc[:, col] == val1) & (df.iloc[:, y].isnull()))
          pass
        elif not pd.isnull(val1) and not pd.isnull(val2):
          count_dict[(val1, val2)] = np.count_nonzero((df.iloc[:, col] == val1) & (df.iloc[:, y] == val2))
      
    for val in y_value_counts.keys():
      if y_value_counts[val] != 0:
        h_y = h_y - y_value_counts[val]/df.iloc[:, y].value_counts().sum()\
        * math.log(y_value_counts[val]/df.iloc[:, y].value_counts().sum())
    
    for val1 in df.iloc[:, col].unique():
      for val2 in df.iloc[:, y].unique():
        if pd.isnull(val1) and pd.isnull(val2):
#           h_x_y = h_x_y - col_value_counts["nan"]/df.shape[0]*count_dict[("nan", "nan")]/col_value_counts["nan"]*\
#           math.log(count_dict[("nan", "nan")]/col_value_counts["nan"])
          pass
        elif pd.isnull(val1) and not pd.isnull(val2):
#           h_x_y = h_x_y - col_value_counts["nan"]/df.shape[0]*count_dict[("nan", val2)]/col_value_counts["nan"]*\
#           math.log(count_dict[("nan", val2)]/col_value_counts["nan"])
          pass
        elif not pd.isnull(val1) and pd.isnull(val2):
#           h_x_y = h_x_y - col_value_counts[val1]/df.shape[0]*count_dict[(val1, "nan")]/col_value_counts[val1]*\
#           math.log(count_dict[(val1, "nan")]/col_value_counts[val1])
          pass
        elif not pd.isnull(val1) and not pd.isnull(val2) and count_dict[(val1, val2)] != 0:
          h_x_y = h_x_y - col_value_counts[val1]/df.iloc[:, col].value_counts().sum()\
      *count_dict[(val1, val2)]/col_value_counts[val1]*\
          math.log(count_dict[(val1, val2)]/col_value_counts[val1])
          
  return h_y - h_x_y

In [11]:
calculate_info_gain_g(df_temp, 0, 2, False)

0.1438410362258904